# Emerging Technologies Problems

In [4]:
# Libraries used in this notebook:
import random
import numpy as np
import itertools as it

## Problem 1: Generating Random Boolean Functions

In [ ]:
def random_constant_balanced():
    

## Problem 2: Classical Testing for Function Type

## Problem 3: Quantum Oracles


## Problem 4: Deutsch's Algorithm with Qiskit

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm